# Imports

In [ ]:
import pandas as pd

from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, SimpleRNN # for creating regular densely-connected NN layers and RNN layers
from keras.layers import Embedding, Dense, LSTM, Dropout

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import math # to help with data reshaping of the data

# Sklearn
import sklearn # for model evaluation
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import mean_squared_error # for model evaluation metrics
from sklearn.preprocessing import MinMaxScaler # for feature scaling
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


Tensorflow/Keras: 2.12.0
pandas: 1.5.3
numpy: 1.22.4
sklearn: 1.2.2


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
%cd "/gdrive/MyDrive/_RESEARCH/UPM/RV Jesus"

Mounted at /gdrive
/gdrive/MyDrive/_RESEARCH/UPM/RV Jesus


# Evaluation

In [ ]:
def concatenate_outputs_of_generation(generator):
  c = 0
  a =None
  for i in generator:
    if c == 0:
      a = i[1]
      c=1
    else:
      a = np.concatenate((a,i[1]))
  return a

def average(lst):
    return sum(lst) / len(lst)

In [ ]:
%pip install pyquaternion
from pyquaternion import Quaternion

SCALE_FACTOR_SCENES = 1.2

def calculate_MDE(y,y_predicted):
  accumulator = 0

  for i in range(y.shape[0]):
    distance = np.linalg.norm((np.array([y[i,0]-y_predicted[i,0],y[i,1]-y_predicted[i,1]], dtype = "float32")),axis=0)
    accumulator += distance

  return (accumulator/y.shape[0])/SCALE_FACTOR_SCENES

def calculate_MDE_quaternion(df,y,y_predicted):
  accumulator = 0

  for i in range(y.shape[0]):
    y_real = Quaternion(df.iloc[i,:4]).rotate([y[i,0],y[i,1],y[i,2]])
    y_pred = Quaternion(df.iloc[i,:4]).rotate([y_predicted[i,0],y_predicted[i,1],y_predicted[i,2]])

    distance = np.linalg.norm((np.array([y_real[0]-y_pred[0],y_real[2]-y_pred[2]], dtype = "float32")),axis=0)
    accumulator += distance

  return (accumulator/y.shape[0])/SCALE_FACTOR_SCENES

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def evaluate_model(model,list_generators, scale=False):
  res=[]
  print('---------- Evaluation on Test Data ----------')
  for a in range(0,len(list_generators)):
      gener=list_generators[a]
      scaler= gener[3]

      test_generator=gener[1]
      pred_test = model.predict_generator(test_generator)
      y_test =  concatenate_outputs_of_generation(test_generator)

      ## Data transform
      pred_test_scaled= pred_test
      y_test_scaled= y_test

      if scale:
        pred_test_scaled= scaler.inverse_transform(pred_test)
        y_test_scaled= scaler.inverse_transform(y_test)

      if model == 'QUAT_3LABEL':
        #print('MDE QUAT: ' + str(calculate_MDE_quaternion(y_test_scaled, pred_test_scaled)), end = "||||")
        res.append( calculate_MDE_quaternion(y_test_scaled, pred_test_scaled))
      else:
        #print('MDE: ' + str(calculate_MDE(y_test_scaled, pred_test_scaled)), end = "||||")
        res.append( calculate_MDE(y_test_scaled, pred_test_scaled))

  print("")
  return res


In [ ]:
import keras.backend as K

def squared_euclidean_distance(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sum(K.square(y_pred - y_true), axis=-1)



In [ ]:
import os

subfolders = ['BASELINE_ORG','QUAT']#,'BASELINE_CIRC','BASE_PLUS']
models = [('sr_short','dfSRshort.pkl'),('mf_short','dfMFshort.pkl'),('sl_short','dfSLshort.pkl'),('all_short','Allshort.pkl')]
#models = [('sr_long','dfSRlong.pkl'),('mf_long','dfMFlong.pkl'),('sl_long','dfSLlong.pkl'),('all_long','Alllong.pkl')]

for model_name, df_name in models:
  for folder in subfolders:
    if os.path.isfile('outputPickles/' + folder + '/models/'+model_name+'.h5'):
      ## Cargamos el modelo
      mymodel = keras.models.load_model('outputPickles/' + folder + '/models/'+model_name+'.h5', custom_objects={'squared_euclidean_distance':squared_euclidean_distance})

      ## Cargamos los generators
      list_generators = ""
      import pickle
      with open("outputPickles/" + folder + "/"+ df_name, "rb") as infile:
        list_generators = pickle.load(infile)

      print(">>>>>>>>>>>>>>>>" + folder + "_" + model_name +"<<<<<<<<<<<<<<<<")
      res = evaluate_model(mymodel, list_generators)#, list_scalers)
      print(">>>>>>>>>> TOTAL MDE: " + folder + "_" + model_name +": "+ str(average(res)), end = "\n\n")
    else:
      print('outputPickles/' + folder + '/models/'+model_name+".h5---> No existe")


>>>>>>>>>>>>>>>>BASELINE_ORG_sr_short<<<<<<<<<<<<<<<<
---------- Evaluation on Test Data ----------


<ipython-input-5-aabaf81fc6b9>:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred_test = model.predict_generator(test_generator)



>>>>>>>>>> TOTAL MDE: BASELINE_ORG_sr_short: 0.01460729804657237

>>>>>>>>>>>>>>>>QUAT_sr_short<<<<<<<<<<<<<<<<
---------- Evaluation on Test Data ----------


<ipython-input-5-aabaf81fc6b9>:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred_test = model.predict_generator(test_generator)



>>>>>>>>>> TOTAL MDE: QUAT_sr_short: 0.009924062431574466

>>>>>>>>>>>>>>>>BASELINE_ORG_mf_short<<<<<<<<<<<<<<<<
---------- Evaluation on Test Data ----------


...


>>>>>>>>>> TOTAL MDE: QUAT_sl_short: 0.02212953670573277

>>>>>>>>>>>>>>>>BASELINE_ORG_all_short<<<<<<<<<<<<<<<<
---------- Evaluation on Test Data ----------


<ipython-input-5-aabaf81fc6b9>:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred_test = model.predict_generator(test_generator)



>>>>>>>>>> TOTAL MDE: BASELINE_ORG_all_short: 0.01874395863813001

>>>>>>>>>>>>>>>>QUAT_all_short<<<<<<<<<<<<<<<<
---------- Evaluation on Test Data ----------


<ipython-input-5-aabaf81fc6b9>:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred_test = model.predict_generator(test_generator)



>>>>>>>>>> TOTAL MDE: QUAT_all_short: 0.014028455753364316

